Use PyTorch, GPU and Colab.

Implement a simple convolutional neural network with training and validation on CIFAR10 dataset, limited to subset of 3k train and 3k test images.
Train the neural network for ~30 epochs with Adam optimizer and batch size equal to 64. Implement validation on batches.
  
Code the following incrementally:
- Prevent overfitting using a) dropout and b) data augmentation.
- Add test augmentation.
- For better results use transfer learning.

In [ ]:
import torch
torch.manual_seed(0)
 
import random
random.seed(0)
 
import numpy as np
np.random.seed(0)
 
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

transform = transforms.Compose( [ transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ] )
 
batch_size = 64
num_workers = 0
path = '~/Downloads/CIFAR10/'
 
def create_loader(path: str, train: bool, transform, download: bool = True) -> torch.utils.data.DataLoader:
    dataset = torchvision.datasets.CIFAR10(root=path, train=train, download=download, transform=transform)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    return dataloader
 
transform_tr =  transform
transform_val = transform
 
trainloader = create_loader(path, train=True, transform=transform_tr)
testloader = create_loader(path, train=False, transform=transform_val)

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# sample data
display_limit = 8
 
def imshow(img):
  img = img / 2 + 0.5     # unnormalize
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1, 2, 0)))
  plt.show()
 
# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)
 
# show images
imshow(torchvision.utils.make_grid(images[:display_limit]))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(display_limit)))

In [ ]:
W, H = 32, 32
Ch = 3
 
# should have conv 32, conv 64, conv 128 (all with kernel size 3x3, stride 2 and padding 1) with relu activations, hidden linear layer 40 with relu, and linear output
class Net(nn.Module): 
    def __init__(self, init_W, init_H, init_Ch):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=init_Ch,
                out_channels=32,
                kernel_size=3,
                stride=2,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=2,
                padding=1,
            ),
            nn.ReLU(),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=2,
                padding=1,
            ),
            nn.ReLU(),
        )

        # fully connected layer, output 10 classes
        self.out = nn.Linear(128 * init_W * init_H, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [ ]:
 # Use Adam optimizer
import torch.optim as optim
import torch.nn.functional as F

net = Net()

In [ ]:
from torch.utils.tensorboard import SummaryWriter
 
epochs = 40
 
def train(epochs, trainloader, net, log_interval):
  writer = SummaryWriter()
  optimizer = optim.Adam(net.parameters())
  
  for epoch in range(epochs):  # loop over the dataset multiple times
    for batch_idx, (data, target) in enumerate(trainloader):
        optimizer.zero_grad()
        output = net(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainloader.dataset),
                100. * batch_idx / len(trainloader), loss.item()))

In [ ]:
def save_net(path, net):
     torch.save(net.state_dict(), path)
 
PATH = './cifar_net.pth'
save_net(PATH, net)

In [ ]:
def load_params(path, net):
    net.load_state_dict(torch.load(path))

load_params(PATH, net)

In [ ]:
def validate(loader, name, net=net):
    net.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            output = net(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(loader.dataset),
        100. * correct / len(loader.dataset)))
 
validate(trainloader, "train")
validate(testloader, "test")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs